In [1]:
import tkinter as tk
from tkinter import font
import math
import re

class ScientificCalculator:
    def __init__(self, root):
        self.root = root
        self.root.title("Scientific Calculator")
        self.root.geometry("400x550")
        self.root.resizable(False, False)
        
        # Custom fonts
        self.display_font = font.Font(size=18, weight='bold')
        self.button_font = font.Font(size=10)
        
        # Variables
        self.current_input = ""
        self.result = ""
        self.radian_mode = True  # Default to radian mode
        
        # Create display
        self.create_display()
        
        # Create buttons
        self.create_buttons()
        
        # Bind keyboard events
        self.root.bind('<Key>', self.key_press)
        
    def create_display(self):
        # Frame for display
        display_frame = tk.Frame(self.root, height=80, bd=3, relief=tk.SUNKEN)
        display_frame.pack(pady=5, padx=5, fill=tk.BOTH)
        
        # Input display
        self.input_display = tk.Label(
            display_frame, 
            text="", 
            anchor=tk.E, 
            font=self.display_font,
            bg="#f0f0f0",
            padx=5
        )
        self.input_display.pack(fill=tk.BOTH, expand=True)
        
        # Result display
        self.result_display = tk.Label(
            display_frame, 
            text="0", 
            anchor=tk.E, 
            font=self.display_font,
            bg="#f0f0f0",
            padx=5
        )
        self.result_display.pack(fill=tk.BOTH, expand=True)
    
    def create_buttons(self):
        # Frame for buttons
        button_frame = tk.Frame(self.root)
        button_frame.pack(padx=5, pady=5, fill=tk.BOTH, expand=True)
        
        # Button layout
        buttons = [
            ["sin", "cos", "tan", "π", "√"],
            ["asin", "acos", "atan", "e", "x²"],
            ["log", "ln", "x^y", "(", ")"],
            ["7", "8", "9", "DEL", "AC"],
            ["4", "5", "6", "*", "/"],
            ["1", "2", "3", "+", "-"],
            ["0", ".", "±", "=", "RAD/DEG"]
        ]
        
        # Create buttons
        for i, row in enumerate(buttons):
            for j, text in enumerate(row):
                btn = tk.Button(
                    button_frame,
                    text=text,
                    font=self.button_font,
                    width=4,
                    height=1,
                    command=lambda t=text: self.on_button_click(t)
                )
                btn.grid(row=i, column=j, padx=2, pady=2, sticky="nsew")
                button_frame.grid_columnconfigure(j, weight=1)
            button_frame.grid_rowconfigure(i, weight=1)
            
        # Configure button colors
        self.configure_button_colors(button_frame)
    
    def configure_button_colors(self, frame):
        # Number buttons
        for num in "0123456789":
            for widget in frame.winfo_children():
                if isinstance(widget, tk.Button) and widget.cget("text") == num:
                    widget.config(bg="#e0e0e0")
        
        # Operator buttons
        for op in ["+", "-", "*", "/", "x^y", "x²"]:
            for widget in frame.winfo_children():
                if isinstance(widget, tk.Button) and widget.cget("text") == op:
                    widget.config(bg="#ffa500")
        
        # Function buttons
        for func in ["sin", "cos", "tan", "asin", "acos", "atan", "log", "ln", "√"]:
            for widget in frame.winfo_children():
                if isinstance(widget, tk.Button) and widget.cget("text") == func:
                    widget.config(bg="#90ee90")
        
        # Special buttons
        for widget in frame.winfo_children():
            text = widget.cget("text")
            if text in ["=", "AC", "DEL", "RAD/DEG"]:
                widget.config(bg="#ff6347")
            elif text in ["(", ")", "π", "e", "±", "."]:
                widget.config(bg="#add8e6")

    def on_button_click(self, text):
        """Handle button clicks"""
        if text == "=":
            self.calculate()
        elif text == "AC":
            self.clear_all()
        elif text == "DEL":
            self.delete_last()
        elif text == "±":
            self.toggle_sign()
        elif text == "RAD/DEG":
            self.toggle_angle_mode()
        elif text in ["sin", "cos", "tan", "asin", "acos", "atan", "log", "ln", "√", "x²", "π", "e", "x^y"]:
            self.handle_functions(text)
        else:
            self.current_input += text
            self.update_display()

    def handle_functions(self, func):
        """Handle scientific functions"""
        if func == "π":
            self.current_input += str(math.pi)
        elif func == "e":
            self.current_input += str(math.e)
        elif func == "x²":
            self.current_input += "²"
        elif func == "x^y":
            self.current_input += "^"
        elif func in ["sin", "cos", "tan", "asin", "acos", "atan", "log", "ln", "√"]:
            self.current_input += f"{func}("
        self.update_display()

    def toggle_sign(self):
        """Toggle positive/negative sign"""
        if self.current_input and self.current_input[-1].isdigit():
            parts = re.split(r'([-+]?\d*\.?\d+)', self.current_input)
            parts = [p for p in parts if p]
            last_num = parts[-1]
            
            if last_num.startswith('-'):
                parts[-1] = last_num[1:]
            else:
                parts[-1] = f"-{last_num}"
            
            self.current_input = ''.join(parts)
            self.update_display()

    def toggle_angle_mode(self):
        """Toggle between radian and degree mode"""
        self.radian_mode = not self.radian_mode
        mode = "RAD" if self.radian_mode else "DEG"
        for widget in self.root.winfo_children():
            if isinstance(widget, tk.Button) and widget.cget("text") == "RAD/DEG":
                widget.config(text=mode)
                break

    def clear_all(self):
        """Clear all inputs"""
        self.current_input = ""
        self.result = ""
        self.update_display()

    def delete_last(self):
        """Delete last character"""
        if self.current_input:
            if self.current_input.endswith(("sin(", "cos(", "tan(", "asin(", "acos(", "atan(", "log(", "ln(", "√(")):
                self.current_input = self.current_input[:-4]
            else:
                self.current_input = self.current_input[:-1]
            self.update_display()

    def update_display(self):
        """Update the display"""
        display_text = self.current_input
        display_text = display_text.replace("^", " ^ ")
        display_text = display_text.replace("²", "²")
        display_text = display_text.replace("*", " × ")
        display_text = display_text.replace("/", " ÷ ")
        
        self.input_display.config(text=display_text)
        
        if self.result:
            self.result_display.config(text=self.result)
        else:
            self.result_display.config(text="0")

    def calculate(self):
        """Perform calculation"""
        try:
            expr = self.current_input
            expr = expr.replace("²", "**2")
            expr = expr.replace("^", "**")
            expr = expr.replace("×", "*")
            expr = expr.replace("÷", "/")
            
            # Handle trigonometric functions
            for func in ["sin", "cos", "tan", "asin", "acos", "atan"]:
                while f"{func}(" in expr:
                    start = expr.find(f"{func}(") + len(func) + 1
                    end = self.find_matching_parenthesis(expr, start-1)
                    
                    arg = expr[start:end]
                    arg_value = eval(arg, {"__builtins__": None}, {"math": math})
                    
                    if not self.radian_mode:
                        if func in ["sin", "cos", "tan"]:
                            arg_value = math.radians(arg_value)
                        elif func in ["asin", "acos", "atan"]:
                            arg_value = math.degrees(arg_value)
                    
                    func_value = getattr(math, func)(arg_value)
                    expr = expr[:expr.find(f"{func}(")] + str(func_value) + expr[end+1:]
            
            # Handle other functions
            expr = expr.replace("ln(", "math.log(")
            expr = expr.replace("log(", "math.log10(")
            expr = expr.replace("√(", "math.sqrt(")
            
            # Evaluate the expression
            self.result = str(eval(expr, {"__builtins__": None}, {"math": math}))
            self.current_input = self.result
            self.update_display()
            
        except Exception as e:
            self.result = "Error"
            self.update_display()

    def find_matching_parenthesis(self, expr, start_pos):
        """Find matching parenthesis"""
        stack = []
        for i in range(start_pos, len(expr)):
            if expr[i] == '(':
                stack.append(i)
            elif expr[i] == ')':
                if stack:
                    stack.pop()
                    if not stack:
                        return i
        return len(expr)

    def key_press(self, event):
        """Handle keyboard input"""
        key = event.char
        keys_mapping = {
            '\r': '=',
            '\x08': 'DEL',
            '\x1b': 'AC',
            'p': 'π',
            'E': 'e'
        }
        
        if key in keys_mapping:
            self.on_button_click(keys_mapping[key])
        elif key in "0123456789+-*/.()":
            self.on_button_click(key)
        elif key.lower() == 's':
            self.on_button_click("sin")
        elif key.lower() == 'c':
            self.on_button_click("cos")
        elif key.lower() == 't':
            self.on_button_click("tan")
        elif key.lower() == 'l':
            self.on_button_click("log")
        elif key.lower() == 'n':
            self.on_button_click("ln")
        elif key == '^':
            self.on_button_click("x^y")
        elif key == '!':
            self.on_button_click("x²")
        elif key == '@':
            self.on_button_click("√")
        elif key == 'a':
            self.on_button_click("asin")
        elif key == 'o':
            self.on_button_click("acos")
        elif key == 'd':
            self.on_button_click("atan")

if __name__ == "__main__":
    root = tk.Tk()
    calculator = ScientificCalculator(root)
    root.mainloop()